In [1]:
import math
from tensorboardX import SummaryWriter
import env_configurations
sess = None

/home/trrrrr/anaconda3/envs/tf14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/trrrrr/anaconda3/envs/tf14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/trrrrr/anaconda3/envs/tf14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/trrrrr/anaconda3/envs/tf14/lib/python3.6/site-

In [2]:
import tensorflow as tf
import collections
from a2c_continuous import A2CAgent
import tr_helpers
import networks

gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers
import games_configurations
env_name = 'FlexAnt'#'RoboschoolHalfCheetah-v1'#'BipedalWalker-v2'#'Pendulum-v0'#'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
#a2c_config = games_configurations.pendulum_lstm_config
#a2c_config = games_configurations.halfcheetah_lstm_config_v2
#a2c_config = games_configurations.roboschoolant_config
#a2c_config = a2c_games_configurations.pendulum_config
#a2c_config = games_configurations.bipedalwalker_config
#a2c_config = games_configurations.roboschoolhumanoid_lstm_config
a2c_config = games_configurations.flexant_config
obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
print(obs_space)
print(action_space)


Box(39,)
Box(8,)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-11-09 14:16:55,846	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-11-09_14-16-55_23770/logs.
2019-11-09 14:16:55,968	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:62053 to respond...
2019-11-09 14:16:56,116	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:47555 to respond...
2019-11-09 14:16:56,120	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-11-09 14:16:56,163	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-11-09_14-16-55_23770/logs.
2019-11-09 14:16:56,165	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.1.86',
 'redis_address': '192.168.1.86:62053',
 'object_store_address': '/tmp/ray/session_2019-11-09_14-16-55_23770/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-11-09_14-16-55_23770/sockets/raylet',
 'webui_url': None}

In [ ]:




agent = A2CAgent(sess,'run3', obs_space, False, action_space, a2c_config)
#agent.restore('nn/latest_run_renderer_best')
agent.train()
#agent.save('nn/latest')

Instructions for updating:
Use keras.layers.dense instead.


Instructions for updating:
Use keras.layers.dense instead.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Frames per seconds:  26131.450806038
saving next best rewards:  -25.133736397176982
Frames per seconds:  26341.2130086795
Frames per seconds:  27324.36639144938
Frames per seconds:  27622.32522942994
Frames per seconds:  27777.6083713424
Frames per seconds:  25330.141387829182
Frames per seconds:  27720.286279483284
Frames per seconds:  29061.490762217618
Frames per seconds:  27966.47039497446
Frames per seconds:  27856.866316223648
Frames per seconds:  24975.629489966468
saving next best rewards:  -23.399328966178
Frames per seconds:  27395.60175493447
saving next best rewards:  -9.000634237984196
Frames per seconds:  25462.24610104072
Frames per seconds:  26166.02692718663
saving next best rewards:  0.5231862856075168
Frames per seconds:  27561.338125187063
saving next best rewards:  5.1681032188422975
Frames per seconds:  27072.455904889546
saving next best rewards:  20.141739298272878
Frames per seconds:  26629.83080550814
Frames per seconds:  27251.503805504
saving next best rewar

In [ ]:
agent.save('nn/latest_run_multirenderer_best')
ray.shutdown()

In [ ]:
import gym
gym.envs.registry.all()